<a href="https://colab.research.google.com/github/jacobpaul07/Delivery_app_android/blob/master/1_Ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! git clone https://github.com/awarebayes/RecNN
! pip install ./RecNN
! pip install gdown
! pip install --upgrade tqdm
! pip install --upgrade pandas
! pip install jupyterthemes

clear_output()

In [0]:
! wget http://files.grouplens.org/datasets/movielens/ml-20m.zip
! gdown https://drive.google.com/uc?id=1EQ_zXBR3DKpmJR3jBgLvt-xoOvArGMsL
! unzip ml-20m.zip


--2020-03-06 07:23:45--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  26.2MB/s    in 7.9s    

2020-03-06 07:23:54 (23.9 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]

Downloading...
From: https://drive.google.com/uc?id=1EQ_zXBR3DKpmJR3jBgLvt-xoOvArGMsL
To: /content/ml20_pca128.pkl
22.7MB [00:00, 227MB/s]
Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


In [0]:
! git clone https://github.com/pandas-dev/pandas.git


Cloning into 'pandas'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 215944 (delta 0), reused 0 (delta 0), pack-reused 215943
Receiving objects: 100% (215944/215944), 177.46 MiB | 28.92 MiB/s, done.
Resolving deltas: 100% (179620/179620), done.
Permission denied: https://drive.google.com/uc?id=1t0LNCbqLjiLkAMFwtP8OIYU-
Maybe you need to change permission over 'Anyone with the link'?
tar: parsed.tar.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [0]:
! gdown https://drive.google.com/uc?id=1t0LNCbqLjiLkAMFwtP8OIYU-zPUCNAjK
! tar -xvf parsed.tar.gz

Downloading...
From: https://drive.google.com/uc?id=1t0LNCbqLjiLkAMFwtP8OIYU-zPUCNAjK
To: /content/parsed.tar.gz
21.6MB [00:00, 127MB/s]
parsed/tmdb.json
parsed/omdb.json
parsed/


In [0]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
import numpy as np
from scipy.spatial import distance

from tqdm.auto import tqdm
import pickle
import gc
import json
import h5py
import pandas as pd

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

# == recnn ==
import sys
sys.path.append("../../")
import recnn


cuda = torch.device('cuda')
frame_size = 10
# https://drive.google.com/open?id=1t0LNCbqLjiLkAMFwtP8OIYU-zPUCNAjK
meta = json.load(open('/content/parsed/omdb.json'))
tqdm.pandas()

ImportError: ignored

In [0]:
env = recnn.data.FrameEnv('/content/ml20_pca128.pkl',
                         '/content/ml-20m/ratings.csv', 10, 1)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
ddpg = recnn.nn.models.Actor(1290, 128, 256).to(cuda)
td3 = recnn.nn.models.Actor(1290, 128, 256).to(cuda)
ddpg.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/ddpg_policy.pt'))
td3.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/td3_policy.pt'))

In [0]:
test_batch = next(iter(env.test_dataloader))
state, action, reward, next_state, done = recnn.data.get_base_batch(test_batch)

In [0]:
def rank(gen_action, metric):
    scores = []
    for i in env.movie_embeddings_key_dict.keys():
        if i == 0 or i == '0':
            continue
        scores.append([i, metric(env.movie_embeddings_key_dict[i], gen_action)])
    scores = list(sorted(scores, key = lambda x: x[1]))
    scores = scores[:10]
    ids = [i[0] for i in scores]
    for i in range(10):
        scores[i].extend([meta[str(scores[i][0])]['omdb'][key]  for key in ['Title',
                                'Genre', 'Language', 'Released', 'imdbRating']])
        # scores[i][3] = ' '.join([genres_dict[i] for i in scores[i][3]])

    indexes = ['id', 'score', 'Title', 'Genre', 'Language', 'Released', 'imdbRating']
    table_dict = dict([(key,[i[idx] for i in scores]) for idx, key in enumerate(indexes)])
    table = pd.DataFrame(table_dict)
    return table

In [0]:
'0' in env.movie_embeddings_key_dict.keys()

# DDPG

In [0]:
ddpg_action = ddpg(state)
# pick random action
ddpg_action = ddpg_action[np.random.randint(0, state.size(0), 1)[0]].detach().cpu().numpy()

In [0]:
!pip install pandas==0.23.1
print(pd.__version__)

In [0]:
from pandas.plotting import table
import subprocess
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(ddpg_action, distance.euclidean)

In [0]:
rank(ddpg_action, distance.cosine)

In [0]:
rank(ddpg_action, distance.correlation) # looks similar to cosine

In [0]:
rank(ddpg_action, distance.canberra)

In [0]:
rank(ddpg_action, distance.minkowski)

In [0]:
rank(ddpg_action, distance.chebyshev)

In [0]:
rank(ddpg_action, distance.braycurtis)

In [0]:
rank(ddpg_action, distance.cityblock)

# TD3

In [0]:
td3_action = td3(state)
# pick random action
td3_action = td3_action[np.random.randint(0, state.size(0), 1)[0]].detach().cpu().numpy()

In [0]:
from pandas.plotting import table 
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(td3_action, distance.euclidean)

In [0]:
rank(td3_action, distance.cosine)

In [0]:
rank(td3_action, distance.correlation) # looks similar to cosine

In [0]:
rank(td3_action, distance.canberra)

In [0]:
rank(td3_action, distance.minkowski)

In [0]:
rank(td3_action, distance.chebyshev)

In [0]:
rank(td3_action, distance.braycurtis)

In [0]:
rank(td3_action, distance.cityblock)

# BCQ

In [0]:
perturbator = recnn.models.bcqPerturbator(1290, 128, 256).to(cuda)
generator = recnn.models.bcqGenerator(1290, 128, 512).to(cuda)

perturbator.load_state_dict(torch.load('../../models/bcq_perturbator.pt'))
generator.load_state_dict(torch.load('../../models/bcq_generator.pt'))

In [0]:
sampled_actions = generator.decode(state)
perturbed_actions= perturbator(state, sampled_actions)

In [0]:
bcq_action = perturbed_actions[np.random.randint(0,
                               sampled_actions.size(0), 1)[0]].detach().cpu().numpy()

In [0]:
rank(bcq_action, distance.euclidean)

In [0]:
rank(bcq_action, distance.cosine)

In [0]:
rank(bcq_action, distance.correlation)

In [0]:
rank(bcq_action, distance.canberra)

In [0]:
rank(bcq_action, distance.minkowski)

In [0]:
rank(bcq_action, distance.chebyshev)

In [0]:
rank(bcq_action, distance.braycurtis)

In [0]:
rank(bcq_action, distance.cityblock)